In [2]:
#

In [3]:



from itertools import product
import logging
import os
import pandas as pd
import pickle
import trader as trader
import importlib
importlib.reload(trader)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [14]:

results_path = "results"


#  data\forecast_actuals_lin_regression_Low_col_5_days.pkl
# data\forecast_actuals_lin_regression_High_col_1_days.pkl
# results\forecast_actuals_lin_regression_High_col_5_days.pkl
# results\forecast_actuals_mlp_regression_High_col_5_days.pkl
# results\forecast_actuals_mlp_regression_Low_col_5_days.pkl

def load_results_pickle_file(model_type, forecast_target_col, forecast_leading_days):
    pickle_file_name = os.path.join(results_path, f"forecast_actuals_{model_type}_{forecast_target_col}_col_{forecast_leading_days}_days.pkl".replace("/", "-"))
    logger.info(f"Loading pickle file: {pickle_file_name}")
    with open(pickle_file_name, 'rb') as file:
        return pickle.load(file)


results = []
model_types = [
    'lin_regression',
    'mlp_regression',
]
forecast_target_cols = [
    'Low',
    'High',
    'Close'
]
forecast_leading_days = [
    1,
    3,
    5
]
# zip the lists together
for model_type, forecast_target_col, forecast_leading_day in product(model_types, forecast_target_cols, forecast_leading_days):
    # logger.info(f" model_type: {model_type}, forecast_target_col: {forecast_target_col}, forecast_leading_day: {forecast_leading_day}")
    df = load_results_pickle_file(model_type, forecast_target_col, forecast_leading_day)
    df['model_type'] = model_type
    results.append(df)
forecast_actuals_df = pd.concat(results, axis=0)
pickle_file_name = os.path.join(results_path, f"forecast_actuals_all_models.pkl")
logger.info(f"Saving pickle file: {pickle_file_name}")
with open(pickle_file_name, 'wb') as file:
        pickle.dump(forecast_actuals_df, file)
forecast_actuals_df



INFO:__main__: model_type: lin_regression, forecast_target_col: Low, forecast_leading_day: 1
INFO:__main__:Loading pickle file: results\forecast_actuals_lin_regression_Low_col_1_days.pkl
INFO:__main__: model_type: lin_regression, forecast_target_col: Low, forecast_leading_day: 3
INFO:__main__:Loading pickle file: results\forecast_actuals_lin_regression_Low_col_3_days.pkl
INFO:__main__: model_type: lin_regression, forecast_target_col: Low, forecast_leading_day: 5
INFO:__main__:Loading pickle file: results\forecast_actuals_lin_regression_Low_col_5_days.pkl
INFO:__main__: model_type: lin_regression, forecast_target_col: High, forecast_leading_day: 1
INFO:__main__:Loading pickle file: results\forecast_actuals_lin_regression_High_col_1_days.pkl
INFO:__main__: model_type: lin_regression, forecast_target_col: High, forecast_leading_day: 3
INFO:__main__:Loading pickle file: results\forecast_actuals_lin_regression_High_col_3_days.pkl
INFO:__main__: model_type: lin_regression, forecast_target_co

,symbol,Date,label_date,Open,Close,High,Low,label_date_reverse,forecast_leading,forecast_target,actual_column_name,forecast_leading_days,model_type
0,AMZN,2023-12-03,2023-12-04,146.000000,147.029999,147.250000,145.550003,NaT,145.839628,NaN,Low,1,lin_regression
1,AMZN,2023-12-04,2023-12-05,145.250000,144.839996,145.350006,142.809998,2023-12-03,143.005282,145.839628,Low,1,lin_regression
2,AMZN,2023-12-05,2023-12-06,143.550003,146.880005,148.570007,143.130005,2023-12-04,145.997800,143.005282,Low,1,lin_regression
3,AMZN,2023-12-06,2023-12-07,147.580002,144.520004,147.850006,144.279999,2023-12-05,142.343156,145.997800,Low,1,lin_regression
4,AMZN,2023-12-07,2023-12-08,146.149994,146.880005,147.919998,145.339996,2023-12-06,144.315381,142.343156,Low,1,lin_regression
...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,ZTS,NaT,NaT,NaN,NaN,NaN,NaN,2024-01-19,NaN,187.407932,Close,5,mlp_regression
423,ZTS,NaT,NaT,NaN,NaN,NaN,NaN,2024-01-20,NaN,187.407932,Close,5,mlp_regression
424,ZTS,NaT,NaT,NaN,NaN,NaN,NaN,2024-01-21,NaN,187.407932,Close,5,mlp_regression
425,ZTS,NaT,NaT,NaN,NaN,NaN,NaN,2024-01-22,NaN,190.670407,Close,5,mlp_regression


In [15]:
import sklearn_util as sklu
import importlib
importlib.reload(sklu)

all_metrics = []
for model_type in model_types:
    df = forecast_actuals_df[forecast_actuals_df['model_type'] == model_type]
    metrics, metrics_str = sklu.calculate_metrics(df, 'forecast_target', 'Low')
    logger.info(f"model_type:{model_type} target col: 'Low' \n {metrics_str}")
    all_metrics.append(metrics)
    metrics, metrics_str = sklu.calculate_metrics(df, 'forecast_target', 'High')
    logger.info(f"model_type:{model_type} target col: 'High' \n {metrics_str}")
    all_metrics.append(metrics)
    
metrics, metrics_str = sklu.calculate_metrics(forecast_actuals_df, 'forecast_target', 'High')
logger.info(metrics_str)
metrics

INFO:__main__:model_type:lin_regression target col: 'Low' 
  MAE: 1.81 MSE: 7.02 RMSE: 2.65 MAPE: 1.93 %
INFO:__main__:model_type:lin_regression target col: 'High' 
  MAE: 1.70 MSE: 6.57 RMSE: 2.56 MAPE: 1.87 %
INFO:__main__:model_type:mlp_regression target col: 'Low' 
  MAE: 2.07 MSE: 8.15 RMSE: 2.85 MAPE: 2.41 %
INFO:__main__:model_type:mlp_regression target col: 'High' 
  MAE: 1.98 MSE: 8.32 RMSE: 2.89 MAPE: 2.30 %
INFO:__main__: MAE: 1.84 MSE: 7.45 RMSE: 2.73 MAPE: 2.09 %


,symbol,Date,label_date,Open,Close,High,Low,label_date_reverse,forecast_leading,forecast_target,actual_column_name,forecast_leading_days,model_type
1,AMZN,2023-12-04,2023-12-05,145.250000,144.839996,145.350006,142.809998,2023-12-03,143.005282,145.839628,Low,1,lin_regression
2,AMZN,2023-12-05,2023-12-06,143.550003,146.880005,148.570007,143.130005,2023-12-04,145.997800,143.005282,Low,1,lin_regression
3,AMZN,2023-12-06,2023-12-07,147.580002,144.520004,147.850006,144.279999,2023-12-05,142.343156,145.997800,Low,1,lin_regression
4,AMZN,2023-12-07,2023-12-08,146.149994,146.880005,147.919998,145.339996,2023-12-06,144.315381,142.343156,Low,1,lin_regression
5,AMZN,2023-12-08,2023-12-09,145.479996,147.419998,147.839996,145.399994,2023-12-07,146.279179,144.315381,Low,1,lin_regression
...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,ZTS,2024-01-19,2024-01-24,190.570007,188.380005,190.919998,187.160004,2024-01-14,187.407932,199.976150,Close,5,mlp_regression
388,ZTS,2024-01-20,2024-01-25,190.570007,188.380005,190.919998,187.160004,2024-01-15,187.407932,199.976150,Close,5,mlp_regression
389,ZTS,2024-01-21,2024-01-26,190.570007,188.380005,190.919998,187.160004,2024-01-16,187.407932,193.997379,Close,5,mlp_regression
390,ZTS,2024-01-22,2024-01-27,190.110001,191.199997,194.250000,190.110001,2024-01-17,190.670407,193.165210,Close,5,mlp_regression


In [16]:
metrics, metrics_str = sklu.calculate_metrics(forecast_actuals_df, 'forecast_target', 'Low')
logger.info(metrics_str)
metrics

INFO:__main__: MAE: 1.94 MSE: 7.58 RMSE: 2.75 MAPE: 2.17 %


,symbol,Date,label_date,Open,Close,High,Low,label_date_reverse,forecast_leading,forecast_target,actual_column_name,forecast_leading_days,model_type
1,AMZN,2023-12-04,2023-12-05,145.250000,144.839996,145.350006,142.809998,2023-12-03,143.005282,145.839628,Low,1,lin_regression
2,AMZN,2023-12-05,2023-12-06,143.550003,146.880005,148.570007,143.130005,2023-12-04,145.997800,143.005282,Low,1,lin_regression
3,AMZN,2023-12-06,2023-12-07,147.580002,144.520004,147.850006,144.279999,2023-12-05,142.343156,145.997800,Low,1,lin_regression
4,AMZN,2023-12-07,2023-12-08,146.149994,146.880005,147.919998,145.339996,2023-12-06,144.315381,142.343156,Low,1,lin_regression
5,AMZN,2023-12-08,2023-12-09,145.479996,147.419998,147.839996,145.399994,2023-12-07,146.279179,144.315381,Low,1,lin_regression
...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,ZTS,2024-01-19,2024-01-24,190.570007,188.380005,190.919998,187.160004,2024-01-14,187.407932,199.976150,Close,5,mlp_regression
388,ZTS,2024-01-20,2024-01-25,190.570007,188.380005,190.919998,187.160004,2024-01-15,187.407932,199.976150,Close,5,mlp_regression
389,ZTS,2024-01-21,2024-01-26,190.570007,188.380005,190.919998,187.160004,2024-01-16,187.407932,193.997379,Close,5,mlp_regression
390,ZTS,2024-01-22,2024-01-27,190.110001,191.199997,194.250000,190.110001,2024-01-17,190.670407,193.165210,Close,5,mlp_regression


In [17]:
import trader as trader
import importlib
importlib.reload(trader)

grouping_dimensions = ['symbol', 'model_type']
forecast_dimensions = ['High', 'Low']
date_col = 'date'
forecast_leading_col = 'forecast_leading'
forecast_target_col = 'forecast_target'
actual_col_name = 'actual_column_name'
forecast_leading_days_col = 'forecast_leading_days'
forecast_leading_days = [1, 2, 3, 4]
file_path = None
df = pd.DataFrame({
    'symbol': ['AAPL', 'AAPL', 'GOOG', 'GOOG'],
    'model_type': ['type1', 'type2', 'type1', 'type2'],
    'forecast': [100, 200, 300, 400],
    'date': pd.to_datetime(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04']),
    'forecast_leading': [1, 2, 3, 4],
    'forecast_target': [100, 200, 300, 400],
    'High': [90, 210, 290, 410],
    'Low': [80, 125, 145, 365],
    'actual_column_name': ['High', 'High', 'Low', 'Low'],
    'forecast_leading_days': [1, 2, 3, 4]
})
test_df = pd.DataFrame({
    'symbol': ['AAPL', 'AAPL'],
    'model_type': ['type1', 'type2'],
    'forecast': [100, 200],
    'date': pd.to_datetime(['2022-01-01', '2022-01-02']),
    'forecast_leading': [1, 2],
    'forecast_target': [100, 200],
    'High': [90, 210],
    'Low': [80, 125],
    'actual_column_name': ['High', 'High'],
    'forecast_leading_days': [1, 2]
})
data_handler = trader.DataHandler(
    grouping_dimensions, 
    forecast_dimensions, 
    date_col,
    forecast_leading_col, 
    forecast_target_col,
    actual_col_name,
    forecast_leading_days_col,
    forecast_leading_days,
    file_path, 
    df
)
data_handler.dataframe

forecast  forecast_leading  forecast_target  \
symbol model_type date                                                      
AAPL   type1      2022-01-01       100                 1              100   
       type2      2022-01-02       200                 2              200   
GOOG   type1      2022-01-03       300                 3              300   
       type2      2022-01-04       400                 4              400   

                              High  Low actual_column_name  \
symbol model_type date                                       
AAPL   type1      2022-01-01    90   80               High   
       type2      2022-01-02   210  125               High   
GOOG   type1      2022-01-03   290  145                Low   
       type2      2022-01-04   410  365                Low   

                              forecast_leading_days  
symbol model_type date                               
AAPL   type1      2022-01-01                      1  
       type2      2022-01-02                      2  
GOOG   type1      2022-01-03                      3  
       type2      2022-01-04                      4

In [18]:
def pivot_forecasts(df_input):
    
    df = df_input.copy(deep=True)

    # Assuming df is your DataFrame
    # Replace 'your_data.csv' with your actual data file
    # df = pd.read_csv('your_data.csv')

    # Creating a unique identifier for each combination
    df['combined_name'] = df['actual_column_name'] + '_' + df['forecast_leading_days'].astype(str)

    # Pivot table for forecast_target
    pivot_target = df.pivot_table(index=['symbol', 'model_type', 'date'], 
                                columns='combined_name', 
                                values='forecast_target',
                                aggfunc='first')

    # Renaming columns for clarity
    pivot_target.columns = [f"forecast_target_{col}" for col in pivot_target.columns]

    # Pivot table for forecast_leading
    pivot_leading = df.pivot_table(index=['symbol', 'model_type', 'date'], 
                                columns='combined_name', 
                                values='forecast_leading',
                                aggfunc='first')

    # Renaming columns for clarity
    pivot_leading.columns = [f"forecast_leading_{col}" for col in pivot_leading.columns]
    pivot_leading
    df_merged = df.merge(pivot_target, on=['symbol', 'model_type', 'date'])


    # Merging the pivot tables into the original DataFrame

    df_merged = df_merged.merge(pivot_leading, on=['symbol', 'model_type', 'date'])
    df_merged
    # Now df_merged will have your desired structure


    # Dropping the now redundant columns
    columns_to_drop = ['forecast_target', 'forecast_leading', 'actual_column_name', 'forecast_leading_days', 'combined_name', 'index', 'label_date', 'label_date_reverse']
    df_merged.drop(columns=columns_to_drop, inplace=True, errors='ignore')
    df_merged
    # Removing duplicate rows
    df_final = df_merged.drop_duplicates()
    return df_final

pivot_forecasts(data_handler.dataframe)


forecast  High  Low  forecast_target_High_1  \
symbol model_type date                                                      
AAPL   type1      2022-01-01       100    90   80                   100.0   
       type2      2022-01-02       200   210  125                     NaN   
GOOG   type1      2022-01-03       300   290  145                     NaN   
       type2      2022-01-04       400   410  365                     NaN   

                              forecast_target_High_2  forecast_target_Low_3  \
symbol model_type date                                                        
AAPL   type1      2022-01-01                     NaN                    NaN   
       type2      2022-01-02                   200.0                    NaN   
GOOG   type1      2022-01-03                     NaN                  300.0   
       type2      2022-01-04                     NaN                    NaN   

                              forecast_target_Low_4  forecast_leading_High_1  \
symbol model_type date                                                         
AAPL   type1      2022-01-01                    NaN                      1.0   
       type2      2022-01-02                    NaN                      NaN   
GOOG   type1      2022-01-03                    NaN                      NaN   
       type2      2022-01-04                  400.0                      NaN   

                              forecast_leading_High_2  forecast_leading_Low_3  \
symbol model_type date                                                          
AAPL   type1      2022-01-01                      NaN                     NaN   
       type2      2022-01-02                      2.0                     NaN   
GOOG   type1      2022-01-03                      NaN                     3.0   
       type2      2022-01-04                      NaN                     NaN   

                              forecast_leading_Low_4  
symbol model_type date                                
AAPL   type1      2022-01-01                     NaN  
       type2      2022-01-02                     NaN  
GOOG   type1      2022-01-03                     NaN  
       type2      2022-01-04                     4.0

In [19]:



import trader as trader
import importlib
importlib.reload(trader)

file_path = 'test/results/forecast_actuals_all_models.pkl'
grouping_dimensions = ['symbol', 'model_type']
forecast_dimensions = ['High', 'Low', 'Close', 'Open']
date_col = 'Date'
forecast_leading_col = 'forecast_leading'
forecast_target_col = 'forecast_target'
actual_col_name = 'actual_column_name'
forecast_leading_days_col = 'forecast_leading_days'
forecast_leading_days = [1, 3, 5]
data_handler = trader.DataHandler(
    grouping_dimensions=grouping_dimensions,
    forecast_dimensions=forecast_dimensions,
    date_col=date_col,
    forecast_leading_col=forecast_leading_col,
    forecast_target_col=forecast_target_col,
    actual_col_name=actual_col_name,
    forecast_leading_days_col = 'forecast_leading_days',
    forecast_leading_days=forecast_leading_days,
    file_path=file_path
)
data_handler

data_handler.dataframe


label_date        Open       Close  \
symbol model_type     Date                                            
AMZN   lin_regression 2023-11-25 2023-11-30  146.699997  146.740005   
                      2023-11-25 2023-11-30  146.699997  146.740005   
                      2023-11-26 2023-12-01  146.699997  146.740005   
                      2023-11-26 2023-12-01  146.699997  146.740005   
                      2023-11-27 2023-11-30  147.529999  147.729996   
...                                     ...         ...         ...   
ZTS    mlp_regression NaT               NaT         NaN         NaN   
                      NaT               NaT         NaN         NaN   
                      NaT               NaT         NaN         NaN   
                      NaT               NaT         NaN         NaN   
                      NaT               NaT         NaN         NaN   

                                        High         Low label_date_reverse  \
symbol model_type     Date                                                    
AMZN   lin_regression 2023-11-25  147.199997  145.320007                NaT   
                      2023-11-25  147.199997  145.320007                NaT   
                      2023-11-26  147.199997  145.320007                NaT   
                      2023-11-26  147.199997  145.320007                NaT   
                      2023-11-27  149.259995  146.880005                NaT   
...                                      ...         ...                ...   
ZTS    mlp_regression NaT                NaN         NaN         2024-01-15   
                      NaT                NaN         NaN         2024-01-16   
                      NaT                NaN         NaN         2024-01-17   
                      NaT                NaN         NaN         2024-01-18   
                      NaT                NaN         NaN         2024-01-19   

                                  forecast_leading  forecast_target  \
symbol model_type     Date                                            
AMZN   lin_regression 2023-11-25        144.377694              NaN   
                      2023-11-25        145.182497              NaN   
                      2023-11-26        144.377694              NaN   
                      2023-11-26        145.182497              NaN   
                      2023-11-27        146.612969              NaN   
...                                            ...              ...   
ZTS    mlp_regression NaT                      NaN       196.642549   
                      NaT                      NaN       193.424375   
                      NaT                      NaN       191.401113   
                      NaT                      NaN       190.175323   
                      NaT                      NaN       190.175323   

                                 actual_column_name  forecast_leading_days  
symbol model_type     Date                                                  
AMZN   lin_regression 2023-11-25                Low                      5  
                      2023-11-25               High                      5  
                      2023-11-26                Low                      5  
                      2023-11-26               High                      5  
                      2023-11-27                Low                      3  
...                                             ...                    ...  
ZTS    mlp_regression NaT                      High                      5  
                      NaT                      High                      5  
                      NaT                      High                      5  
                      NaT                      High                      5  
                      NaT                      High                      5  

[4788 rows x 10 columns]

In [20]:

single_row_df = data_handler.pivot_forecast_dimensions()
single_row_df


,symbol,model_type,Date,Open,Close,High,Low,forecast_target_High_1,forecast_target_High_3,forecast_target_High_5,forecast_target_Low_1,forecast_target_Low_3,forecast_target_Low_5,forecast_leading_High_1,forecast_leading_High_3,forecast_leading_High_5,forecast_leading_Low_1,forecast_leading_Low_3,forecast_leading_Low_5
0,AMZN,lin_regression,2023-11-30,144.759995,146.089996,146.929993,144.330002,148.535574,149.726382,145.182497,144.816794,146.612969,144.377694,148.374341,148.843338,146.045240,143.775565,145.215954,145.807624
6,AMZN,lin_regression,2023-12-01,146.000000,147.029999,147.250000,145.550003,148.374341,146.708033,145.182497,143.775565,143.480264,144.377694,148.603602,150.354977,148.205711,145.277715,147.110412,148.479505
12,AMZN,lin_regression,2023-12-02,146.000000,147.029999,147.250000,145.550003,148.603602,147.668444,146.262457,145.277715,144.510129,146.150255,148.603602,150.354977,148.205711,145.277715,147.110412,148.479505
18,AMZN,lin_regression,2023-12-03,146.000000,147.029999,147.250000,145.550003,148.603602,148.843338,145.893449,145.277715,145.215954,145.762562,148.603602,150.354977,148.205711,145.277715,147.110412,148.479505
24,AMZN,lin_regression,2023-12-04,145.250000,144.839996,145.350006,142.809998,148.603602,150.354977,145.458257,145.277715,147.110412,144.808447,145.969511,147.437034,145.989233,143.180429,143.791588,145.618261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4254,ZTS,mlp_regression,2024-01-15,197.679993,198.940002,199.559998,196.869995,199.972841,199.357175,197.220305,197.098439,194.077078,198.038724,199.972841,199.357175,196.642549,197.098439,194.077078,197.791368
4260,ZTS,mlp_regression,2024-01-16,198.110001,192.990005,198.919998,192.669998,199.972841,199.357175,195.814311,197.098439,194.077078,196.790075,194.626186,195.681823,193.424375,189.398318,189.276646,193.221145
4266,ZTS,mlp_regression,2024-01-17,191.990005,190.899994,193.020004,189.710007,194.626186,199.357175,196.642549,189.398318,194.077078,197.791368,191.504341,193.110918,191.401113,188.120267,187.411512,191.684535
4272,ZTS,mlp_regression,2024-01-18,191.429993,190.500000,192.815002,189.169998,191.504341,199.357175,196.642549,188.120267,194.077078,197.791368,191.195341,193.195248,190.175323,187.570342,187.431715,190.933083


In [21]:
import sklearn_util as sklu
import importlib
importlib.reload(sklu)

sklu.draw_forecast_and_actuals(single_row_df)

In [24]:
import datetime
import trader as trader
import importlib
importlib.reload(trader)

data_handler = trader.DataHandler(
    grouping_dimensions=['symbol', 'model_type'],
    forecast_dimensions=['High', 'Low', 'Close', 'Open'],
    date_col='Date',
    forecast_leading_col='forecast_leading',
    forecast_target_col='forecast_target',
    actual_col_name='actual_column_name',
    forecast_leading_days_col='forecast_leading_days',
    forecast_leading_days=[1, 3, 5],
    file_path='test/results/forecast_actuals_all_models.pkl'
    )

start_date = datetime.datetime.strptime('2024-01-01', '%Y-%m-%d')
default_model_type = 'lin_regression'
portfolio = trader.Portfolio(initial_cash=10000)
trade_simulator = trader.TradeSimulator(portfolio=portfolio, data_handler=data_handler)
strategy_simulator = trader.StrategySimulator(trade_simulator=trade_simulator, data_handler=data_handler, model_type=default_model_type)

strategy_simulator.simulate_strategy(start_date)
strategy_simulator.simulate_trades(start_date)
print("Portfolio Cash: ", portfolio.get_cash())
print("Portfolio Cost: ", portfolio.get_average_cost())
print("Portfolio Value: ", portfolio.get_total_current_value())
for symbol, holding in portfolio.holdings.items():
    print(f"Portfolio Symbol: {symbol} Shares: {holding[0]} Price: {holding[1]} Value: {holding[1] * holding[0]}")
print("Trading History: ")
for order_log in trade_simulator.get_transaction_log():
    print(f" order: {order_log[0].to_string()} time:{order_log[1]}")


Portfolio Cash:  9921.505315248849
Portfolio Cost:  9.333320826898438
Portfolio Value:  9930.838636075747
Portfolio Symbol: VZ Shares: 0.02726041537460361 Price: 48.910869074062425 Value: 1.3333306072917956
Portfolio Symbol: GRMN Shares: 0.008554891160258014 Price: 155.85615911027023 Value: 1.3333324778442173
Portfolio Symbol: ZTS Shares: 0.005594163055065518 Price: 238.3435664624924 Value: 1.3333327739170278
Portfolio Symbol: VICI Shares: 0.0349687368022822 Price: 38.12919648766479 Value: 1.333329836459653
Portfolio Symbol: GOOG Shares: 0.007346835167816104 Price: 181.48394079816526 Value: 1.3333325986498163
Portfolio Symbol: AMZN Shares: 0.006953804304252949 Price: 191.74146691724357 Value: 1.333332637952903
Portfolio Symbol: BKR Shares: 0.03438550307911671 Price: 38.77593099961926 Value: 1.3333298947830252
Trading History: 
 order: Order(symbol=VZ, operation=buy, quantity=0.028093139317679032, price=47.461072575122834) time:2024-01-01 00:00:00
 order: Order(symbol=GRMN, operation=bu

In [23]:
import datetime
import trader as trader
import importlib
importlib.reload(trader)

data_handler = trader.DataHandler(
    grouping_dimensions=['symbol', 'model_type'],
    forecast_dimensions=['High', 'Low', 'Close', 'Open'],
    date_col='Date',
    forecast_leading_col='forecast_leading',
    forecast_target_col='forecast_target',
    actual_col_name='actual_column_name',
    forecast_leading_days_col='forecast_leading_days',
    forecast_leading_days=[1, 3, 5],
    file_path='test/results/forecast_actuals_all_models.pkl'
    )

start_date = datetime.datetime.strptime('2024-01-01', '%Y-%m-%d')
default_model_type = 'lin_regression'
portfolio = trader.Portfolio(initial_cash=10000)
trade_simulator = trader.TradeSimulator(portfolio=portfolio, data_handler=data_handler)
strategy_simulator = trader.StrategySimulator(trade_simulator=trade_simulator, data_handler=data_handler, model_type=default_model_type)

strategy_simulator.simulate_strategy(start_date)
strategy_simulator.simulate_trades(start_date)
print("Portfolio Cash: ", portfolio.get_cash())
print("Portfolio Cost: ", portfolio.get_average_cost())
print("Portfolio Value: ", portfolio.get_total_current_value())
for symbol, holding in portfolio.holdings.items():
    print(f"Portfolio Symbol: {symbol} Shares: {holding[0]} Price: {holding[1]} Value: {holding[1] * holding[0]}")
print("Trading History: ")
for order_log in trade_simulator.get_transaction_log():
    print(f" order: {order_log[0].to_string()} time:{order_log[1]}")

--- Logging error ---
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\miniconda3\Lib\logging\__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\miniconda3\Lib\logging\__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\miniconda3\Lib\logging\__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\ProgramData\miniconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\ProgramData\miniconda3\Lib\site-packages\traitl

AttributeError: 'tuple' object has no attribute 'quantity'

In [ ]:
import trader as trader
import importlib
importlib.reload(trader)

initial_cash = 10000  # Starting cash for the portfolio
portfolio = trader.Portfolio(initial_cash)
simulator = trader.StrategySimulator(handler, portfolio)

simulation_results = simulator.simulate_strategy('GOOG', initial_cash, 'lin_regression', price_per_trade=1)
print(simulation_results)
print("Portfolio Value: ", portfolio.get_value())

[('GOOG', 'buy', 0.0073313786271787685, 130.6300048828125), ('GOOG', 'buy', 0.0073313786271787685, 132.38999938964844), ('GOOG', 'buy', 0.0073313786271787685, 131.42999267578125), ('GOOG', 'sell', 0.0073313786271787685, 138.4499969482422), ('GOOG', 'buy', 0.0073313786271787685, 136.63999938964844), ('GOOG', 'buy', 0.0073313786271787685, 136.63999938964844), ('GOOG', 'sell', 0.0073313786271787685, 136.63999938964844), ('GOOG', 'sell', 0.0073313786271787685, 134.6999969482422), ('GOOG', 'sell', 0.0073313786271787685, 133.63999938964844), ('GOOG', 'sell', 0.0073313786271787685, 133.97000122070312), ('GOOG', 'buy', 0.0073313786271787685, 133.1999969482422), ('GOOG', 'sell', 0.0073313786271787685, 133.83999633789062), ('GOOG', 'buy', 0.0073313786271787685, 133.83999633789062), ('GOOG', 'buy', 0.0073313786271787685, 137.19000244140625), ('GOOG', 'buy', 0.0073313786271787685, 138.10000610351562), ('GOOG', 'buy', 0.0073313786271787685, 139.66000366210938), ('GOOG', 'buy', 0.0073313786271787685

In [ ]:
import trader as trader
import importlib
importlib.reload(trader)

initial_cash = 10000  # Starting cash for the portfolio
portfolio = trader.Portfolio(initial_cash)
simulator = trader.StrategySimulator(handler, portfolio)

simulation_results = simulator.simulate_strategy('VICI', initial_cash, 'lin_regression', price_per_trade=10)
print(simulation_results)
print("Portfolio Value: ", portfolio.get_value())

[('VICI', 'buy', 0.3376097135944177, 29.6200008392334), ('VICI', 'buy', 0.3376097135944177, 29.889999389648438), ('VICI', 'buy', 0.3376097135944177, 30.5), ('VICI', 'buy', 0.3376097135944177, 30.5), ('VICI', 'buy', 0.3376097135944177, 30.5), ('VICI', 'buy', 0.3376097135944177, 30.549999237060547), ('VICI', 'sell', 0.3376097135944177, 30.260000228881836), ('VICI', 'buy', 0.3376097135944177, 30.239999771118164), ('VICI', 'buy', 0.3376097135944177, 30.43000030517578), ('VICI', 'buy', 0.3376097135944177, 30.510000228881836), ('VICI', 'buy', 0.3376097135944177, 30.510000228881836), ('VICI', 'buy', 0.3376097135944177, 30.510000228881836), ('VICI', 'buy', 0.3376097135944177, 30.790000915527344), ('VICI', 'buy', 0.3376097135944177, 30.729999542236328), ('VICI', 'buy', 0.3376097135944177, 31.530000686645508), ('VICI', 'sell', 0.3376097135944177, 31.93000030517578), ('VICI', 'sell', 0.3376097135944177, 31.34000015258789), ('VICI', 'sell', 0.3376097135944177, 31.34000015258789), ('VICI', 'buy', 0

self.dataframe[actual_col_name].dtype object


In [ ]:
# data_handler.dataframe.groupby(level=data_handler.grouping_dimensions + ['Date']).size().max()

6

In [ ]:
# mask = data_handler.dataframe[forecast_leading_days_col].isin(forecast_leading_days)
# mask = mask & (data_handler.dataframe[actual_col_name].isin(forecast_dimensions))
# dataframe = data_handler.dataframe[mask]
# dataframe


index label_date        Open       Close  \
symbol model_type     Date                                                   
AMZN   lin_regression 2023-11-25      0 2023-11-30  146.699997  146.740005   
                      2023-11-25      0 2023-11-30  146.699997  146.740005   
                      2023-11-26      1 2023-12-01  146.699997  146.740005   
                      2023-11-26      1 2023-12-01  146.699997  146.740005   
                      2023-11-27      0 2023-11-30  147.529999  147.729996   
...                                 ...        ...         ...         ...   
ZTS    mlp_regression NaT           422        NaT         NaN         NaN   
                      NaT           423        NaT         NaN         NaN   
                      NaT           424        NaT         NaN         NaN   
                      NaT           425        NaT         NaN         NaN   
                      NaT           426        NaT         NaN         NaN   

                                        High         Low label_date_reverse  \
symbol model_type     Date                                                    
AMZN   lin_regression 2023-11-25  147.199997  145.320007                NaT   
                      2023-11-25  147.199997  145.320007                NaT   
                      2023-11-26  147.199997  145.320007                NaT   
                      2023-11-26  147.199997  145.320007                NaT   
                      2023-11-27  149.259995  146.880005                NaT   
...                                      ...         ...                ...   
ZTS    mlp_regression NaT                NaN         NaN         2024-01-15   
                      NaT                NaN         NaN         2024-01-16   
                      NaT                NaN         NaN         2024-01-17   
                      NaT                NaN         NaN         2024-01-18   
                      NaT                NaN         NaN         2024-01-19   

                                  forecast_leading  forecast_target  \
symbol model_type     Date                                            
AMZN   lin_regression 2023-11-25        144.377694              NaN   
                      2023-11-25        145.182497              NaN   
                      2023-11-26        144.377694              NaN   
                      2023-11-26        145.182497              NaN   
                      2023-11-27        146.612969              NaN   
...                                            ...              ...   
ZTS    mlp_regression NaT                      NaN       196.642549   
                      NaT                      NaN       193.424375   
                      NaT                      NaN       191.401113   
                      NaT                      NaN       190.175323   
                      NaT                      NaN       190.175323   

                                 actual_column_name  forecast_leading_days  
symbol model_type     Date                                                  
AMZN   lin_regression 2023-11-25                Low                      5  
                      2023-11-25               High                      5  
                      2023-11-26                Low                      5  
                      2023-11-26               High                      5  
                      2023-11-27                Low                      3  
...                                             ...                    ...  
ZTS    mlp_regression NaT                      High                      5  
                      NaT                      High                      5  
                      NaT                      High                      5  
                      NaT                      High                      5  
                      NaT                      High                      5  

[4788 rows x 11 columns]

In [ ]:

import datetime
import pandas as pd
import trader
import importlib
importlib.reload(trader)

dataframe = pd.DataFrame({
            'symbol': ['AAPL', 'AAPL', 'GOOG', 'GOOG'],
            'model_type': ['type1', 'type1', 'type1', 'type1'],
            'forecast': [100, 200, 300, 400],
            'date': pd.to_datetime(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04']),
            'forecast_leading': [1, 2, 3, 4],
            'forecast_target': [100, 200, 300, 400],
            'High': [90, 210, 290, 410],
            'Low': [80, 125, 145, 365],
            'Close': [85, 150, 180, 400],
            'actual_column_name': ['High', 'High', 'Low', 'Low'],
            'forecast_leading_days': [1, 2, 3, 4]
        })
data_handler = trader.DataHandler(
    grouping_dimensions=['symbol', 'model_type'],
    forecast_dimensions=['High', 'Low', 'Close'],
    date_col='date',
    forecast_leading_col='forecast_leading',
    forecast_target_col='forecast_target',
    actual_col_name='actual_column_name',
    forecast_leading_days_col='forecast_leading_days',
    forecast_leading_days=[1, 3, 5],
    file_path=None, 
    df=dataframe,
)
start_date = datetime.datetime(2021, 1, 1)
# default_model_type = 'lin_regression'
portfolio = trader.Portfolio(initial_cash=10000)
trade_simulator = trader.TradeSimulator(portfolio=portfolio, data_handler=data_handler)
strategy_simulator = trader.StrategySimulator(data_handler=data_handler, trade_simulator=trade_simulator, model_type='type1')

date = datetime.datetime(2021, 1, 1)
unique_symbols = ['AAPL']
data_by_symbol = {
    'AAPL': {
        date: pd.DataFrame({
        'symbol': ['AAPL'],
        'model_type': ['type1'],
        'date': [date],
        'forecast_leading_High_1': [200],
        'forecast_leading_Low_1': [150],
        'High': [180],
        'Low': [120],
        'Close': [160]
        })
    }
}  

strategy_simulator.simulate_strategy_for_date(unique_symbols, data_by_symbol, date)
df = strategy_simulator.simulate_trades(start_date)

# data = trade_simulator.data_handler.get_stock_date_indexed(symbol='AAPL', model_type='type1')
# data = data[data[trade_simulator.data_handler.date_col] == start_date]
# price = data['Close'].values[0]
# order = trader.Order('AAPL', 'buy', 10, price)
# trade_simulator.submit_order(order, start_date)  
# df = trade_simulator.simulate_trades(start_date, model_type='type1')


df

INFO:trader:processing order: <trader.Order object at 0x00000266CEF507D0> submission_date: 2021-01-01 00:00:00 trade_date: 2021-01-01 00:00:00
INFO:trader:get_stock_data_date_indexed: df:                     symbol model_type       date  High  Low  Close  \
symbol model_type                                                    
AAPL   type1      0   AAPL      type1 2021-01-01    90   80     85   

                     forecast_target_High_1  forecast_leading_High_1  \
symbol model_type                                                      
AAPL   type1      0                   100.0                      1.0   

                     forecast_target_Low_3  forecast_leading_Low_3  
symbol model_type                                                   
AAPL   type1      0                    NaN                     NaN  
INFO:trader:get_stock_data_date_indexed: df.reset_index():   symbol model_type       date  High  Low  Close  forecast_target_High_1  \
0   AAPL      type1 2021-01-01    90   80 

,,,symbol,model_type,date,High,Low,Close,forecast_target_High_1,forecast_leading_High_1,forecast_target_Low_3,forecast_leading_Low_3
symbol,model_type,,,,,,,,,,,
AAPL,type1,0,AAPL,type1,2021-01-01,90,80,85,100.0,1.0,NaN,NaN


In [ ]:
df['Low'].values[0]

80

In [ ]:
df[0]

160